# Vícenásobná lineární regrese metodou MLE

Úloha zaměřená na implementaci vícenásobné lineární regrese pomocí metody MLE za předpokladu, že data pochází z normálního rozdělení (viz prezentace).

**S výhodou využijete kód z předchozích cvičení.**

Uměle vytvořená data reprezentují cenu bytů v závislosti na rozměru obytné plochy a vzdálenosti od centra města.

Nainstalujte si balíky plotly a pandas, např:

`pip install plotly pandas`

Import knihoven:

In [9]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

Vygenerování dat:

In [10]:
np.random.seed(0)

n_samples = 200
living_space_m2 = np.random.uniform(50, 150, n_samples)
distance_km = np.random.uniform(0, 12, n_samples)

gen_std = 1_000_000 # Směrodatná odchylka pro generování gaussovského šumu v datech.
price_czk = 100_000 * living_space_m2 - 300_000 * distance_km + np.random.normal(0, gen_std, n_samples)

data = pd.DataFrame({
    "Obytná plocha [m²]": living_space_m2,
    "Vzdálenost od centra [km]": distance_km,
    "Cena [Kč]": price_czk
})


Pro další výpočty využívejte proměnné `n_samples`, `living_space_m2`, `distance_km` a `price_czk`.

Interkativní graf:

In [11]:
def show_data(theta=None, point=None):
    fig = px.scatter_3d(data, x="Obytná plocha [m²]", y="Vzdálenost od centra [km]", z="Cena [Kč]", title="Ceny bytů").update_layout(height=600)

    if theta is not None:
        x = np.linspace(np.min(living_space_m2), np.max(living_space_m2), 2)
        y = np.linspace(np.min(distance_km), np.max(distance_km), 2)

        xx, yy = np.meshgrid(x, y)
        zz = theta[0] + theta[1] * xx + theta[2] * yy

        fig.add_trace(go.Surface(
            x=xx, y=yy, z=zz,
            opacity=0.6,
            showscale=False
        ))

    if point is not None:
        fig.add_trace(go.Scatter3d(
            x=np.array([point[0]]),
            y=np.array([point[1]]),
            z=np.array([point[2]]),
            mode="markers",
            marker=dict(size=10, color="red"),
            name="Predikce"
        ))

    fig.show()

In [12]:
show_data()

Funkce pro odhad parametrů theta pomocí metody MLE **za předpokladu, že data pochází z normálního rozdělení (viz prezentace)**:

In [13]:
def computeThetaMLE(x, y, lmbd=0):
    """
    x - matice vstupních hodnot rozměru (počet vstupních proměnných X počet trénovacích vzorků)
    y - vektor výstupních hodnot (cena bytu)
    lmbd - regularizační parametr lambda
    
    """
    
    #################################################################
    # ZDE DOPLNIT
    
    #pridani sloupce pro bias
    X = np.hstack((np.ones((x.shape[0], 1)), x))
    
    #regularizace
    I = np.eye(X.shape[1])
    I[0, 0] = 0
    
    #theta = (X.T @ X + lmbd * I)^-1 @ X.T @ y -> obdobne jako u 02 cviceni lin regrese
    theta = np.linalg.inv(X.T @ X + lmbd * I) @ X.T @ y
    
    #################################################################
    
    return theta


Linearní regrese bez regularizace:

In [16]:
theta = computeThetaMLE(np.c_[living_space_m2, distance_km], price_czk)
print(theta)

[-371015.30537259  102559.39326447 -298252.352519  ]


Vykreslení výsledků regrese:

In [17]:
show_data(theta)

Určete předpokládanou cenu bytu o ploše 80 m² vzdáleného 4 km od centra:

In [18]:
x_living_space_m2 = 80
x_distance_km = 4

#################################################################
# ZDE DOPLNIT
#x_price_czk, x_living_space_m2 a x_distance_km musi byt cisla!

x_price_czk = theta[0] + theta[1] * x_living_space_m2 + theta[2] * x_distance_km

#################################################################

print(x_price_czk)
show_data(theta, point=(x_living_space_m2, x_distance_km, x_price_czk))

6640726.745709105


Vypočítejte rozptyl a z něj směrodatnou odchylku regresního modelu:

In [19]:
#################################################################
# ZDE DOPLNIT
# Vypočítejte rozptyl do proměnné `var` a směrodatnou odchylku do proměnné `std`.

y_pred = theta[0] + theta[1] * living_space_m2 + theta[2] * distance_km
residuals = price_czk - y_pred

var = np.mean(residuals**2)
std = np.sqrt(var)

#################################################################

print(f"var = {var}")
print(f"std = {std}")

var = 961950499031.206
std = 980790.7519095019


Všimněte si, že směrodatná odchylka modelu se blíží směrodatné odchylce nastavené pro generování gaussovského šumu v umělých datech.